In [1]:
import os

In [2]:
%pwd

'D:\\Desktop\\Superteams AI\\Task 1 Steps to Monitoring DSPy-Qdrant Powered RAG with Prometheus or Grafana\\RAG-Stack-using-DSPy-Qdrant-Mistral-7B\\Research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'D:\\Desktop\\Superteams AI\\Task 1 Steps to Monitoring DSPy-Qdrant Powered RAG with Prometheus or Grafana\\RAG-Stack-using-DSPy-Qdrant-Mistral-7B'

In [5]:
import logging
from pathlib import Path

In [6]:
# Configure the logger
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [7]:
import pyarrow.parquet as pq

# Read the Parquet file into a pyarrow Table
table = pq.read_table("Dataset/train-00000-of-00001-a77e2814210655f1.parquet")

# Convert the Table to a pandas DataFrame
df = table.to_pandas()

# Now you can work with the DataFrame as usual
df.head()


,instruction,input,output,__index_level_0__
0,"If you are a doctor, please answer the medical...",hi. im a home health aide and i have a client ...,"hi, thanks for contacting chatbot. swelling in...",137213
1,Please summerize the given abstract to a title,RATIONALE: The COVID-19 pandemic struck an imm...,Hydroxychloroquine vs. Azithromycin for Hospit...,67540
2,Please summerize the given abstract to a title,Objectives: To investigate the experience of p...,Playing the harmonica with chronic obstructive...,109854
3,"If you are a doctor, please answer the medical...","Hi, Im sorry to bother you but I have fpund a ...",welcome to chatbot .1. the history suggest a p...,137309
4,"If you are a doctor, please answer the medical...","Hi, my penis has a slightly lighter spot at t...","hello, skin color changes, discoloration etc. ...",135382


In [8]:
# Create dfcopy with 100 rows
dfcopy = df.head(15).copy()

# Display the first few rows
dfcopy.head()

,instruction,input,output,__index_level_0__
0,"If you are a doctor, please answer the medical...",hi. im a home health aide and i have a client ...,"hi, thanks for contacting chatbot. swelling in...",137213
1,Please summerize the given abstract to a title,RATIONALE: The COVID-19 pandemic struck an imm...,Hydroxychloroquine vs. Azithromycin for Hospit...,67540
2,Please summerize the given abstract to a title,Objectives: To investigate the experience of p...,Playing the harmonica with chronic obstructive...,109854
3,"If you are a doctor, please answer the medical...","Hi, Im sorry to bother you but I have fpund a ...",welcome to chatbot .1. the history suggest a p...,137309
4,"If you are a doctor, please answer the medical...","Hi, my penis has a slightly lighter spot at t...","hello, skin color changes, discoloration etc. ...",135382


In [9]:
dfcopy = dfcopy.drop("__index_level_0__", axis=1)
dfcopy.head()

,instruction,input,output
0,"If you are a doctor, please answer the medical...",hi. im a home health aide and i have a client ...,"hi, thanks for contacting chatbot. swelling in..."
1,Please summerize the given abstract to a title,RATIONALE: The COVID-19 pandemic struck an imm...,Hydroxychloroquine vs. Azithromycin for Hospit...
2,Please summerize the given abstract to a title,Objectives: To investigate the experience of p...,Playing the harmonica with chronic obstructive...
3,"If you are a doctor, please answer the medical...","Hi, Im sorry to bother you but I have fpund a ...",welcome to chatbot .1. the history suggest a p...
4,"If you are a doctor, please answer the medical...","Hi, my penis has a slightly lighter spot at t...","hello, skin color changes, discoloration etc. ..."


In [10]:
# from transformers import AutoTokenizer, AutoModel
# from pathlib import Path

# def download_model_and_tokenizer(model_name, save_path):
#     """
#     Download and save both the model and the tokenizer to the specified directory.

#     Parameters:
#         model_name (str): Name of the model to download.
#         save_path (str or Path): Path to the directory where the model and tokenizer will be saved.
#     """
#     # Create the save path if it doesn't exist
#     save_path = Path(save_path)
#     save_path.mkdir(parents=True, exist_ok=True)
    
#     # Initialize tokenizer and model
#     tokenizer = AutoTokenizer.from_pretrained(model_name)
#     model = AutoModel.from_pretrained(model_name)
    
#     # Save tokenizer
#     tokenizer.save_pretrained(save_path)
    
#     # Save model
#     model.save_pretrained(save_path)

# # Example usage
# model_name = 'sentence-transformers/all-MiniLM-L12-v2'  # Model name to download
# save_path = Path("MiniLM-L12-v2/")  # Path where model and tokenizer will be saved
# download_model_and_tokenizer(model_name, save_path)


In [11]:
from transformers import AutoTokenizer, AutoModel

def load_model_and_tokenizer(model_path):
    """
    Load the model and tokenizer from the specified directory.

    Parameters:
        model_path (str or Path): Path to the directory containing the saved model and tokenizer.

    Returns:
        tokenizer (transformers.PreTrainedTokenizer): Loaded tokenizer.
        model (transformers.PreTrainedModel): Loaded model.
    """
    model_path = Path(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModel.from_pretrained(model_path)
    return tokenizer, model

# Load the model and tokenizer
model_path = Path("MiniLM-L12-v2/")
tokenizer, model = load_model_and_tokenizer(model_path)

D:\Desktop\Superteams AI\Task 1 Steps to Monitoring DSPy-Qdrant Powered RAG with Prometheus or Grafana\RAG-Stack-using-DSPy-Qdrant-Mistral-7B\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
import torch

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def generate_embedding(text):
    # Tokenize input text
    encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    # Compute token embeddings with model
    with torch.no_grad():
        model_output = model(**encoded_input)
    # Perform mean pooling
    sentence_embedding = mean_pooling(model_output, encoded_input['attention_mask'])
    # Convert to numpy for FAISS compatibility and ensure it's 2D
    return sentence_embedding.cpu().numpy().reshape(1, -1)

In [13]:
dfcopy['merged_data'] = dfcopy.apply(lambda row: row['input'] + ' ' + row['output'], axis=1)
dfcopy.head()

,instruction,input,output,merged_data
0,"If you are a doctor, please answer the medical...",hi. im a home health aide and i have a client ...,"hi, thanks for contacting chatbot. swelling in...",hi. im a home health aide and i have a client ...
1,Please summerize the given abstract to a title,RATIONALE: The COVID-19 pandemic struck an imm...,Hydroxychloroquine vs. Azithromycin for Hospit...,RATIONALE: The COVID-19 pandemic struck an imm...
2,Please summerize the given abstract to a title,Objectives: To investigate the experience of p...,Playing the harmonica with chronic obstructive...,Objectives: To investigate the experience of p...
3,"If you are a doctor, please answer the medical...","Hi, Im sorry to bother you but I have fpund a ...",welcome to chatbot .1. the history suggest a p...,"Hi, Im sorry to bother you but I have fpund a ..."
4,"If you are a doctor, please answer the medical...","Hi, my penis has a slightly lighter spot at t...","hello, skin color changes, discoloration etc. ...","Hi, my penis has a slightly lighter spot at t..."


In [14]:
# Generate embeddings for the 'input' column
dfcopy['embeddings'] = dfcopy['merged_data'].apply(lambda x: generate_embedding(x)[0].tolist())
dfcopy.head()

,instruction,input,output,merged_data,embeddings
0,"If you are a doctor, please answer the medical...",hi. im a home health aide and i have a client ...,"hi, thanks for contacting chatbot. swelling in...",hi. im a home health aide and i have a client ...,"[-0.13907790184020996, -0.18952248990535736, 0..."
1,Please summerize the given abstract to a title,RATIONALE: The COVID-19 pandemic struck an imm...,Hydroxychloroquine vs. Azithromycin for Hospit...,RATIONALE: The COVID-19 pandemic struck an imm...,"[-0.02158217504620552, -0.14091074466705322, -..."
2,Please summerize the given abstract to a title,Objectives: To investigate the experience of p...,Playing the harmonica with chronic obstructive...,Objectives: To investigate the experience of p...,"[0.08616306632757187, 0.04367256164550781, 0.0..."
3,"If you are a doctor, please answer the medical...","Hi, Im sorry to bother you but I have fpund a ...",welcome to chatbot .1. the history suggest a p...,"Hi, Im sorry to bother you but I have fpund a ...","[0.022234562784433365, -0.16184261441230774, 0..."
4,"If you are a doctor, please answer the medical...","Hi, my penis has a slightly lighter spot at t...","hello, skin color changes, discoloration etc. ...","Hi, my penis has a slightly lighter spot at t...","[0.005746546201407909, 0.000758152105845511, -..."


In [15]:
df.head()

,instruction,input,output,__index_level_0__
0,"If you are a doctor, please answer the medical...",hi. im a home health aide and i have a client ...,"hi, thanks for contacting chatbot. swelling in...",137213
1,Please summerize the given abstract to a title,RATIONALE: The COVID-19 pandemic struck an imm...,Hydroxychloroquine vs. Azithromycin for Hospit...,67540
2,Please summerize the given abstract to a title,Objectives: To investigate the experience of p...,Playing the harmonica with chronic obstructive...,109854
3,"If you are a doctor, please answer the medical...","Hi, Im sorry to bother you but I have fpund a ...",welcome to chatbot .1. the history suggest a p...,137309
4,"If you are a doctor, please answer the medical...","Hi, my penis has a slightly lighter spot at t...","hello, skin color changes, discoloration etc. ...",135382


In [16]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

# Initialize Qdrant client
qdrant_client = QdrantClient(host='localhost', port=6333)
collection_name = "medical_dataset"

# Specify the vectors' configuration
vectors_config = VectorParams(
    size=model.config.hidden_size,  # The size of your embeddings
    distance=Distance.COSINE  # The distance metric for the vector space
)

# Create or recreate the collection with the specified configuration
qdrant_client.recreate_collection(
    collection_name=collection_name,
    vectors_config=vectors_config,
    # Optionally, you can specify other parameters for the collection
)



INFO:httpx:HTTP Request: DELETE http://localhost:6333/collections/medical_dataset "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/medical_dataset "HTTP/1.1 200 OK"


True

In [17]:
 # Insert data into Qdrant
for index, row in dfcopy.iterrows():
    qdrant_client.upsert(
        collection_name=collection_name,
        points=[{
            "id": index,  # Using the dataframe index as the ID
            "vector": row['embeddings'],
            "payload": {
                "instruction": row['instruction'], 
                "input": row['input'],
                "output": row['output']
            }
        }]
    )


INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/medical_dataset/points?wait=true "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/medical_dataset/points?wait=true "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/medical_dataset/points?wait=true "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/medical_dataset/points?wait=true "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/medical_dataset/points?wait=true "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/medical_dataset/points?wait=true "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/medical_dataset/points?wait=true "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/medical_dataset/points?wait=true "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/medical_dataset/points?wa

In [18]:
logger.info(f"qdrant '{collection_name}' collection created")

INFO:__main__:qdrant 'medical_dataset' collection created


In [19]:
from langchain.vectorstores import Qdrant

url = "http://localhost:6333"  # URL where the Qdrant service is running
collection_name =  "medical_dataset"  # Name of the collection in Qdrant

# Initialize the Qdrant client with the specified URL
client = QdrantClient(
    url=url,
    prefer_grpc=False  # Indicates whether to use gRPC for communication
)

logger.info(f"QdrantClient initialized: {client}")  # Prints the client information
logger.info(f"#################################")  # Prints a separator line

# Create a Qdrant object with the specified client, embeddings, and collection name
# Initialize the Qdrant vector store from langchain
db = Qdrant(
    client=client,
    embeddings=dfcopy['embeddings'].tolist(),  # Use the generated embeddings
    collection_name=collection_name
)

logger.info(f"Qdrant vector store initialized: {db}") # Prints the database object information

INFO:__main__:QdrantClient initialized: <qdrant_client.qdrant_client.QdrantClient object at 0x00000286019F5720>
INFO:__main__:#################################
D:\Desktop\Superteams AI\Task 1 Steps to Monitoring DSPy-Qdrant Powered RAG with Prometheus or Grafana\RAG-Stack-using-DSPy-Qdrant-Mistral-7B\venv\lib\site-packages\langchain_community\vectorstores\qdrant.py:150: UserWarning: `embeddings` should be an instance of `Embeddings`.Using `embeddings` as `embedding_function` which is deprecated
  warnings.warn(
INFO:__main__:Qdrant vector store initialized: <langchain_community.vectorstores.qdrant.Qdrant object at 0x0000028602E0A2C0>


In [24]:
def similarity_search_with_score(query, k=5):
    query_embedding = generate_embedding(query)[0].tolist()
    search_results = qdrant_client.search(
        collection_name=collection_name,
        query_vector=query_embedding,
        limit=k,
        with_payload=True,
        with_vectors=False
    )
    return search_results

query = "What is Azithromycin?"
search_results = similarity_search_with_score(query=query, k=5)

for result in search_results:
    doc_id = result.id
    score = result.score
    payload = result.payload  # The payload should contain your text or a reference to it.

    # Assuming the payload contains a field 'input' where the text is stored
    doc_content = payload.get('input', 'No content available')

    # Print the similarity score and document content
    logger.info({"score": score, "doc_id": doc_id, "content": doc_content})

INFO:httpx:HTTP Request: POST http://localhost:6333/collections/medical_dataset/points/search "HTTP/1.1 200 OK"
INFO:__main__:{'score': 0.41175747, 'doc_id': 1, 'content': 'RATIONALE: The COVID-19 pandemic struck an immunologically naïve, globally interconnected population. In the face of a new infectious agent causing acute respiratory failure for which there were no known effective therapies, rapid, often pragmatic trials were necessary to evaluate potential treatments, frequently starting with medications that are already marketed for other indications. Early in the pandemic, hydroxychloroquine and azithromycin were two such candidates. OBJECTIVE: Assess the relative efficacy of hydroxychloroquine and azithromycin among hospitalized patients with COVID-19. METHODS: We performed a randomized clinical trial of hydroxychloroquine vs. azithromycin among hospitalized patients with COVID-19. Treatment was 5 days of study medication. The primary endpoint was the COVID Ordinal Outcomes scal